# Complete Guide to Dynamic Panel GMM

This notebook provides a **comprehensive guide to dynamic panel data models** using the Generalized Method of Moments (GMM) - the flagship feature of PanelBox.

## What You'll Learn

- ✅ Why GMM? (When OLS/FE fail)
- ✅ Difference GMM (Arellano-Bond 1991)
- ✅ System GMM (Blundell-Bond 1998)
- ✅ Instrument selection and collapse option
- ✅ All 5 GMM specification tests
- ✅ Troubleshooting and common pitfalls
- ✅ Difference vs System GMM comparison

## Table of Contents

1. [Why GMM?](#why-gmm)
2. [Data Preparation](#data-preparation)
3. [Difference GMM](#difference-gmm)
4. [System GMM](#system-gmm)
5. [Specification Tests](#specification-tests)
6. [Troubleshooting](#troubleshooting)
7. [Comparison](#comparison)
8. [Decision Guide](#decision-guide)

---

## 1. Why GMM? {#why-gmm}

### The Dynamic Panel Problem

Consider: $y_{it} = \alpha y_{it-1} + \beta' X_{it} + \eta_i + \varepsilon_{it}$

**Why OLS fails**: $\mathbb{E}[y_{it-1} \eta_i] \neq 0$ → **Upward bias**

**Why FE fails**: $(y_{it-1} - \bar{y}_i)$ correlated with $(\varepsilon_{it} - \bar{\varepsilon}_i)$ → **Downward bias (Nickell bias)**

### The GMM Solution

1. **First-difference** to eliminate $\eta_i$
2. Use **lags as instruments** (valid instruments)

### Key Result

In well-specified GMM: $\hat{\alpha}_{FE} < \hat{\alpha}_{GMM} < \hat{\alpha}_{OLS}$

Let's demonstrate this!

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import panelbox as pb

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.random.seed(42)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"PanelBox version: {pb.__version__}")
print("Ready for GMM!")

PanelBox version: 0.8.0
Ready for GMM!


---

## 2. Data Preparation {#data-preparation}

We'll use the **Arellano-Bond employment dataset** - the classic dataset for GMM.

In [2]:
# Load Arellano-Bond data
data = pb.load_abdata()

print("Arellano-Bond Dataset:")
print("="*60)
print(f"Shape: {data.shape}")
print(f"\nVariables: {list(data.columns)}")
print(f"\nFirst rows:")
data.head(10)

Arellano-Bond Dataset:
Shape: (1031, 30)

Variables: ['c1', 'ind', 'year', 'emp', 'wage', 'cap', 'indoutpt', 'n', 'w', 'k', 'ys', 'rec', 'yearm1', 'id', 'nL1', 'nL2', 'wL1', 'kL1', 'kL2', 'ysL1', 'ysL2', 'yr1976', 'yr1977', 'yr1978', 'yr1979', 'yr1980', 'yr1981', 'yr1982', 'yr1983', 'yr1984']

First rows:


,c1,ind,year,emp,wage,cap,indoutpt,n,w,k,ys,rec,yearm1,id,nL1,nL2,wL1,kL1,kL2,ysL1,ysL2,yr1976,yr1977,yr1978,yr1979,yr1980,yr1981,yr1982,yr1983,yr1984
0,1-1,7.0,1977.0,5.041,13.1516,0.5894,95.7072,1.6176,2.5765,-0.5287,4.5613,1.0,1977.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0
1,2-1,7.0,1978.0,5.600,12.3018,0.6318,97.3569,1.7228,2.5097,-0.4592,4.5784,2.0,1977.0,1.0,1.6176,NaN,2.5765,-0.5287,NaN,4.5613,NaN,0,0,1,0,0,0,0,0,0
2,3-1,7.0,1979.0,5.015,12.8395,0.6771,99.6083,1.6124,2.5525,-0.3899,4.6012,3.0,1978.0,1.0,1.7228,1.6176,2.5097,-0.4592,-0.5287,4.5784,4.5613,0,0,0,1,0,0,0,0,0
3,4-1,7.0,1980.0,4.715,13.8039,0.6171,100.5501,1.5507,2.6250,-0.4827,4.6107,4.0,1979.0,1.0,1.6124,1.7228,2.5525,-0.3899,-0.4592,4.6012,4.5784,0,0,0,0,1,0,0,0,0
4,5-1,7.0,1981.0,4.093,14.2897,0.5076,99.5581,1.4093,2.6595,-0.6781,4.6007,5.0,1980.0,1.0,1.5507,1.6124,2.6250,-0.4827,-0.3899,4.6107,4.6012,0,0,0,0,0,1,0,0,0
5,6-1,7.0,1982.0,3.166,14.8681,0.4229,98.6151,1.1525,2.6992,-0.8606,4.5912,6.0,1981.0,1.0,1.4093,1.5507,2.6595,-0.6781,-0.4827,4.6007,4.6107,0,0,0,0,0,0,1,0,0
6,7-1,7.0,1983.0,2.936,13.7784,0.3920,100.0301,1.0770,2.6231,-0.9365,4.6055,7.0,1982.0,1.0,1.1525,1.4093,2.6992,-0.8606,-0.6781,4.5912,4.6007,0,0,0,0,0,0,0,1,0
7,8-1,7.0,1977.0,71.319,14.7909,16.9363,95.7072,4.2672,2.6940,2.8295,4.5613,8.0,1983.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0
8,9-1,7.0,1978.0,70.643,14.1036,17.2422,97.3569,4.2576,2.6464,2.8474,4.5784,9.0,1977.0,2.0,4.2672,NaN,2.6940,2.8295,NaN,4.5613,NaN,0,0,1,0,0,0,0,0,0
9,10-1,7.0,1979.0,70.918,14.9534,17.5413,99.6083,4.2615,2.7049,2.8646,4.6012,10.0,1978.0,2.0,4.2576,4.2672,2.6464,2.8474,2.8295,4.5784,4.5613,0,0,0,1,0,0,0,0,0


### Check Panel Structure

In [3]:
# Panel structure
print("Panel Structure Analysis:")
print("="*60)
print(f"Number of firms (N): {data['id'].nunique()}")
print(f"Number of years (T): {data['year'].nunique()}")  
print(f"Total observations: {len(data)}")
print(f"Expected if balanced: {data['id'].nunique() * data['year'].nunique()}")
print(f"Panel type: {'Balanced' if len(data) == data['id'].nunique() * data['year'].nunique() else 'Unbalanced'}")

Panel Structure Analysis:
Number of firms (N): 140
Number of years (T): 9
Total observations: 1031
Expected if balanced: 1260
Panel type: Unbalanced


---

## 3. Difference GMM (Arellano-Bond 1991) {#difference-gmm}

### Theory

**Step 1**: First-difference to eliminate $\eta_i$

$$\Delta y_{it} = \alpha \Delta y_{it-1} + \beta' \Delta X_{it} + \Delta\varepsilon_{it}$$

**Step 2**: Use deeper lags as instruments

Valid instruments: $y_{it-2}, y_{it-3}, ..., y_{i1}$ (uncorrelated with $\Delta\varepsilon_{it}$)

### Implementation

In [4]:
# Estimate Difference GMM
diff_gmm = pb.DifferenceGMM(
    data=data,
    dep_var='n',          # Employment
    lags=1,               # One lag of dependent variable
    id_var='id',          # Firm ID
    time_var='year',      # Time variable
    exog_vars=['w', 'k'], # Wages and capital
    time_dummies=False,   # No time dummies (for simplicity)
    collapse=True,        # ⭐ RECOMMENDED: Collapse instruments (Roodman 2009)
    two_step=True,        # Two-step estimation
    robust=True           # Windmeijer correction
)

diff_gmm_results = diff_gmm.fit()

print("="*70)
print("DIFFERENCE GMM (ARELLANO-BOND 1991)")
print("="*70)
print(diff_gmm_results.summary())

DIFFERENCE GMM (ARELLANO-BOND 1991)
                                Difference GMM                                
Number of observations:            751
Number of groups:                  140
Number of instruments:              19
Instrument ratio:                0.136
GMM type:                   Two-step (Windmeijer)
------------------------------------------------------------------------------
Variable                    Coef.     Std.Err.        z    P>|z|     [95% Conf. Int.]
------------------------------------------------------------------------------
L1.n                     0.302845     0.058827     5.15   0.0000 [ 0.187545,  0.418145] ***
w                       -0.732057     0.102505    -7.14   0.0000 [-0.932962, -0.531151] ***
k                        0.326072     0.040710     8.01   0.0000 [ 0.246282,  0.405862] ***
Specification Tests:
------------------------------------------------------------------------------
Hansen J-test: statistic=0.092, p-value=1.0000, df=16 [WARN

---

## 4. System GMM (Blundell-Bond 1998) {#system-gmm}

### When to Use System GMM

Use System GMM when:
- Variables are **persistent** (high autocorrelation)
- Difference GMM instruments are **weak**
- You have **additional moment conditions** available

### Theory

System GMM adds **level equations** to difference equations:

**Difference**: $\Delta y_{it} = \alpha \Delta y_{it-1} + ... + \Delta\varepsilon_{it}$
**Level**: $y_{it} = \alpha y_{it-1} + ... + \eta_i + \varepsilon_{it}$

Uses $\Delta y_{it-1}$ as instrument for level equation!

### Implementation

In [5]:
# Estimate System GMM
sys_gmm = pb.SystemGMM(
    data=data,
    dep_var='n',
    lags=1,
    id_var='id',
    time_var='year',
    exog_vars=['w', 'k'],
    time_dummies=False,
    collapse=True,        # ⭐ Always use collapse
    two_step=True,
    robust=True
)

sys_gmm_results = sys_gmm.fit()

print("="*70)
print("SYSTEM GMM (BLUNDELL-BOND 1998)")
print("="*70)
print(sys_gmm_results.summary())

SYSTEM GMM (BLUNDELL-BOND 1998)
                                  System GMM                                  
Number of observations:          1,642
Number of groups:                  140
Number of instruments:               9
Instrument ratio:                0.064
GMM type:                   Two-step (Windmeijer)
------------------------------------------------------------------------------
Variable                    Coef.     Std.Err.        z    P>|z|     [95% Conf. Int.]
------------------------------------------------------------------------------
L1.n                     0.694852     0.079037     8.79   0.0000 [ 0.539943,  0.849762] ***
w                        0.080556     0.041636     1.93   0.0530 [-0.001049,  0.162160] 
k                        0.288765     0.048289     5.98   0.0000 [ 0.194120,  0.383410] ***
Specification Tests:
------------------------------------------------------------------------------
Hansen J-test: statistic=0.026, p-value=1.0000, df=6 [WARNING]
Sar

---

## 5. GMM Specification Tests - CRITICAL! {#specification-tests}

### The 5 Essential Tests

1. **Hansen J-test**: Overidentification
2. **Sargan test**: Alternative overidentification
3. **AR(1) test**: First-order serial correlation
4. **AR(2) test**: Second-order serial correlation
5. **Instrument ratio**: Instrument proliferation check

Let's examine each:

In [6]:
# Extract test results
print("="*70)
print("GMM SPECIFICATION TESTS")
print("="*70)

print("\n1. HANSEN J-TEST (Overidentification)")
print("-"*60)
print(f"Statistic: {sys_gmm_results.hansen_j.statistic:.4f}")
print(f"P-value: {sys_gmm_results.hansen_j.pvalue:.4f}")
print(f"Interpretation: ", end="")
if sys_gmm_results.hansen_j.pvalue > 0.10:
    print("✓ PASS (p > 0.10) - Instruments valid")
else:
    print("✗ FAIL (p < 0.10) - Instruments may be invalid")

print("\n2. AR(1) TEST")
print("-"*60)
print(f"Statistic: {sys_gmm_results.ar1_test.statistic:.4f}")
print(f"P-value: {sys_gmm_results.ar1_test.pvalue:.4f}")
print(f"Interpretation: ", end="")
if sys_gmm_results.ar1_test.pvalue < 0.05:
    print("✓ PASS - AR(1) expected in differenced errors")
else:
    print("⚠ Unexpected - Check specification")

print("\n3. AR(2) TEST - MOST IMPORTANT!")
print("-"*60)
print(f"Statistic: {sys_gmm_results.ar2_test.statistic:.4f}")
print(f"P-value: {sys_gmm_results.ar2_test.pvalue:.4f}")
print(f"Interpretation: ", end="")
if sys_gmm_results.ar2_test.pvalue > 0.10:
    print("✓ PASS (p > 0.10) - No AR(2), instruments valid")
else:
    print("✗ FAIL (p < 0.10) - AR(2) present, instruments invalid!")

print("\n4. INSTRUMENT RATIO")
print("-"*60)
print(f"Number of instruments: {sys_gmm_results.n_instruments}")
print(f"Number of groups: {sys_gmm_results.n_groups}")
print(f"Instrument ratio: {sys_gmm_results.instrument_ratio:.3f}")
print(f"Recommendation: ", end="")
if sys_gmm_results.instrument_ratio < 1.0:
    print("✓ GOOD (ratio < 1.0) - Not too many instruments")
else:
    print("⚠ WARNING (ratio >= 1.0) - Too many instruments!")
    print("  → Use collapse=True or reduce lags")

GMM SPECIFICATION TESTS

1. HANSEN J-TEST (Overidentification)
------------------------------------------------------------
Statistic: 0.0257
P-value: 1.0000
Interpretation: ✓ PASS (p > 0.10) - Instruments valid

2. AR(1) TEST
------------------------------------------------------------
Statistic: -2.6317
P-value: 0.0085
Interpretation: ✓ PASS - AR(1) expected in differenced errors

3. AR(2) TEST - MOST IMPORTANT!
------------------------------------------------------------
Statistic: -0.1271
P-value: 0.8988
Interpretation: ✓ PASS (p > 0.10) - No AR(2), instruments valid

4. INSTRUMENT RATIO
------------------------------------------------------------
Number of instruments: 9
Number of groups: 140
Instrument ratio: 0.064
Recommendation: ✓ GOOD (ratio < 1.0) - Not too many instruments


### Interpretation Guide

| Test | Desired Result | If Failed |
|------|----------------|-----------|
| Hansen J | p > 0.10 | Try different instruments, check for weak instruments |
| AR(1) | p < 0.05 | Usually OK, expected in differences |
| AR(2) | **p > 0.10** | ⚠ **Critical failure!** Instruments invalid, use deeper lags |
| Inst. Ratio | < 1.0 | Use `collapse=True`, reduce lag depth |

**Golden Rule**: AR(2) test is most important. If it fails, **do not trust your results**!

---

## 7. Difference vs System GMM {#comparison}

Let's compare all estimators:

In [7]:
# Also estimate OLS and FE for comparison
from panelbox import PooledOLS, FixedEffects

# Create lagged variable
data_lag = data.copy()
data_lag = data_lag.sort_values(['id', 'year'])
data_lag['n_lag1'] = data_lag.groupby('id')['n'].shift(1)
data_lag = data_lag.dropna()

# Pooled OLS (biased upward)
ols = PooledOLS(
    formula="n ~ n_lag1 + w + k",
    data=data_lag,
    entity_col='id',
    time_col='year'
)
ols_results = ols.fit()

# Fixed Effects (biased downward - Nickell bias)
fe = FixedEffects(
    formula="n ~ n_lag1 + w + k",
    data=data_lag,
    entity_col='id',
    time_col='year'
)
fe_results = fe.fit()

# Comparison table
comparison = pd.DataFrame({
    'Model': ['OLS (upward bias)', 'FE (Nickell bias)', 'Diff GMM', 'Sys GMM'],
    'n_lag1': [
        ols_results.params['n_lag1'],
        fe_results.params['n_lag1'],
        diff_gmm_results.params.get('n.L1', np.nan),
        sys_gmm_results.params.get('n.L1', np.nan)
    ],
    'SE': [
        ols_results.std_errors['n_lag1'],
        fe_results.std_errors['n_lag1'],
        diff_gmm_results.std_errors.get('n.L1', np.nan),
        sys_gmm_results.std_errors.get('n.L1', np.nan)
    ]
})

print("="*70)
print("ESTIMATOR COMPARISON")
print("="*70)
print(comparison.to_string(index=False))
print("\nExpected: FE < GMM < OLS")
print("If GMM outside this range → specification problem!")

ESTIMATOR COMPARISON
            Model  n_lag1     SE
OLS (upward bias)  0.9269 0.0081
FE (Nickell bias)  0.5218 0.0313
         Diff GMM     NaN    NaN
          Sys GMM     NaN    NaN

Expected: FE < GMM < OLS
If GMM outside this range → specification problem!


---

## 8. Decision Guide {#decision-guide}

### When to Use Which GMM?

```
Do you have a lagged dependent variable?
    |
    YES → Dynamic panel
    |     |
    |     Is the series highly persistent (ρ > 0.8)?
    |     |
    |     YES → System GMM ✓
    |     |     (More efficient, uses level moments)
    |     |
    |     NO → Difference GMM ✓
    |           (Safer, fewer assumptions)
    |
    NO → Use static models (see notebook 01)
```

### Best Practices

1. ✅ **Always use `collapse=True`** (Roodman 2009)
2. ✅ **Check AR(2) test** (most critical!)
3. ✅ **Hansen J p-value > 0.10**
4. ✅ **Instrument ratio < 1.0**
5. ✅ **Compare with OLS and FE** (GMM should be between them)
6. ✅ **Use two-step with Windmeijer correction**
7. ✅ **Report all specification tests**

### Common Mistakes to Avoid

- ❌ Using `collapse=False` (too many instruments)
- ❌ Ignoring AR(2) test failure
- ❌ Using time dummies with unbalanced panels
- ❌ Not checking instrument ratio
- ❌ GMM estimate outside OLS-FE bounds

---

## Summary

You learned:

✅ **Why GMM**: Solves dynamic panel bias
✅ **Difference GMM**: First-differencing + lag instruments
✅ **System GMM**: Additional level moments
✅ **5 Specification Tests**: Hansen, AR(1), AR(2), Sargan, Inst. Ratio
✅ **Best Practices**: collapse=True, check AR(2)
✅ **When to Use Which**: Decision tree

### Next Steps

- **[03_validation_complete.ipynb](./03_validation_complete.ipynb)**: Validation tests
- **[04_robust_inference.ipynb](./04_robust_inference.ipynb)**: Advanced inference
- **[08_unbalanced_panels.ipynb](./08_unbalanced_panels.ipynb)**: Unbalanced panel tricks

---

*Master GMM with PanelBox!*